## Helpers 

In [4]:
def read_and_display_logs(log_path):
    """Reads and displays the content of the log file."""
    import os

    if os.path.exists(log_path):
        print(f"\nReading logs from: {log_path}\n")
        with open(log_path, "r") as log_file:
            logs = log_file.read()
            print(logs)
    else:
        print(f"Log file does not exist: {log_path}")

# Example usage
log_path = "work/app/logs/app.log"


In [5]:
import openai
import os

openai.api_key = os.environ["OPENAI_API_KEY"]

models = openai.models.list()
for model in models.data:
    print(model.id)

gpt-5-search-api
gpt-5-search-api-2025-10-14
dall-e-2
gpt-4-turbo-2024-04-09
gpt-audio
gpt-4o-mini-tts
gpt-4-turbo
gpt-realtime
gpt-realtime-2025-08-28
gpt-4.1-mini
gpt-4.1-mini-2025-04-14
gpt-3.5-turbo-1106
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-4-0613
gpt-4
whisper-1
gpt-4.1
gpt-4.1-2025-04-14
text-embedding-ada-002
dall-e-3
gpt-4.1-nano
gpt-3.5-turbo-instruct-0914
o1-2024-12-17
gpt-4.1-nano-2025-04-14
gpt-3.5-turbo-16k
gpt-audio-2025-08-28
gpt-3.5-turbo-instruct
gpt-4o-2024-11-20
gpt-4o-2024-05-13
gpt-4o-mini-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-search-preview
omni-moderation-latest
o1-pro
o1-pro-2025-03-19
gpt-4o-search-preview-2025-03-11
text-embedding-3-small
o1-mini-2024-09-12
gpt-4o-2024-08-06
o1
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-mini-audio-preview
gpt-image-1-mini
gpt-5-mini
gpt-image-1
gpt-5-mini-2025-08-07
omni-moderation-2024-09-26
gpt-5
gpt-5-nano-2025-08-07
gpt-4o-audio-preview-2024-12-17
gpt-5-nano
tts-1-1106
tts-1-hd-1106
tts-1-hd
tts-

### Full Debate 

In [6]:
# Read from volumes: CHARACTER_DUMP_PATH and DEBATE_CONFIG_PATH
import json

# Get paths from environment variables
character_dump_path = "/home/jovyan/work/characters"
debate_config_path = "/home/jovyan/work/configs/debate_config.json"

# Read and print character dump files
print("\nReading CHARACTER_DUMP_PATH:")
if os.path.exists(character_dump_path):
    for file_name in os.listdir(character_dump_path):
        file_path = os.path.join(character_dump_path, file_name)
        if os.path.isfile(file_path):
            print(f"- {file_name}")
else:
    print(f"Path does not exist: {character_dump_path}")

# Read and print debate config file
print("\nReading DEBATE_CONFIG_PATH:")
if os.path.exists(debate_config_path):
    with open(debate_config_path, "r") as f:
        debate_config = json.load(f)
        print(json.dumps(debate_config, indent=4))
else:
    print(f"Path does not exist: {debate_config_path}")


Reading CHARACTER_DUMP_PATH:
- bd91656ec9ae.json
- e21b1dba641c.json
- 0af0871192d0.json
- 70e046e7d721.json

Reading DEBATE_CONFIG_PATH:
{
    "character_creation_prompt": [
        "You are a character creation assistant for app where LLMs will debate eachother",
        "You will be provided a description of a character provided by a user",
        "You must generate a json string with the following format: {'name': 'name': '', 'debate_style': '', 'personality_description': '', 'extra_details': ''}",
        "Do not generate any other text other than the json string it must be parasable directly in python",
        "You may not be provided all details directly, you must infer them based on what a user inputs, do not make major changes",
        "If your character is a known figure search the internet or fill in the json with known information of the character, still respecting other inputs from the user",
        "Your outputs will be input into another LLM please format all respon

In [7]:
# Install if needed
# !pip install langgraph langchain langchain-openai python-dotenv

import os
from typing import TypedDict, Annotated, Sequence, Literal
from langchain_core.messages import BaseMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
import operator

# or
# from dotenv import load_dotenv
# load_dotenv()

In [8]:
model_name = "gpt-3.5-turbo-1106"

In [9]:
# Define the debate state
class DebateState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    turn_count: int
    max_turns: int
    current_speaker: str

# Initialize LLM
llm = ChatOpenAI(model=model_name, temperature=0.7)

In [10]:
def create_debater(name: str, personality: str, topic: str):
    """Create a debater function with specific personality"""
    
    def debate_turn(state: DebateState):
        messages = [
            SystemMessage(content=f"""You are {name}.
            Personality: {personality}
            Topic: {topic}
            
            Make a concise argument (2-3 sentences).
            Respond to previous points if any.
            Stay in character."""),
            *state["messages"]
        ]
        
        response = llm.invoke(messages)
        print(f"\n{name}: {response.content}")
        
        return {
            "messages": [AIMessage(content=f"{name}: {response.content}")],
            "turn_count": state["turn_count"] + 1
        }
    
    return debate_turn

def should_continue(state: DebateState) -> Literal["continue", "end"]:
    """Check if debate should continue"""
    return "end" if state["turn_count"] >= state["max_turns"] else "continue"

def judge_summary(state: DebateState):
    """Judge provides final verdict"""
    messages = [
        SystemMessage(content="""You are an impartial judge. 
        Provide a brief verdict on who won this debate and why.
        Keep it to 3-4 sentences."""),
        *state["messages"]
    ]
    
    response = llm.invoke(messages)
    print(f"\n🔨 VERDICT: {response.content}")
    
    return {"messages": [AIMessage(content=f"Judge: {response.content}")]}

In [12]:
import json
import os

# Load character data
characters = []
if os.path.exists(character_dump_path):
    for file_name in os.listdir(character_dump_path):
        file_path = os.path.join(character_dump_path, file_name)
        if os.path.isfile(file_path):
            with open(file_path, "r") as f:
                character_data = json.load(f)
                characters.append(character_data)
else:
    print(f"Path does not exist: {character_dump_path}")



In [ ]:
characters = characters[:2]

In [17]:
characters

[{'name': 'Ahmad El Sharaa',
  'debate_style': 'Diplomatic and assertive, focusing on finding common ground through respectful dialogue.',
  'personality_description': 'A former Syrian politician and diplomat, known for his experience in international relations and conflict resolution.',
  'extra_details': 'Ahmad El Sharaa was a Syrian politician who served as the Deputy Prime Minister from 1988 to 1992 and later as the Foreign Minister from 1992 to 1994. He played a crucial role in the Syrian peace negotiations with Israel in the 1990s. As a diplomat, he is known for his calm and composed demeanor, often adopting a conciliatory tone in negotiations. During debates, he is expected to maintain a respectful and professional attitude, actively listening to opposing viewpoints and seeking to understand the underlying concerns. He should employ a clear and concise speaking style, using formal language and avoiding aggressive tone. His vocabulary should reflect his experience in internationa

In [20]:
agent1

<function __main__.create_debater.<locals>.debate_turn(state: __main__.DebateState)>

In [19]:
# Set up the debate parameters


topic = "Should AI development be accelerated or slowed down?"

agent1 = create_debater(
    name=characters[0]["name"],
    personality=characters[0]["personality_description"],
    topic=topic
)

agent2 = create_debater(
    name=characters[1]["name"],
    personality=characters[1]["personality_description"],
    topic=topic
)

# Build the graph
workflow = StateGraph(DebateState)

# Add nodes
workflow.add_node("agent1", agent1)
workflow.add_node("agent2", agent2)
workflow.add_node("judge", judge_summary)

# Define flow
workflow.set_entry_point("agent1")
workflow.add_conditional_edges("agent1", should_continue, {
    "continue": "agent2",
    "end": "judge"
})
workflow.add_conditional_edges("agent2", should_continue, {
    "continue": "agent1", 
    "end": "judge"
})
workflow.add_edge("judge", END)

# Compile
graph = workflow.compile()

In [21]:
# Run the debate
print(f"TOPIC: {topic}")
print("=" * 50)

initial_state = {
    "messages": [],
    "turn_count": 0,
    "max_turns": 6,  # 3 rounds each
    "current_speaker": "agent1"
}

result = graph.invoke(initial_state)

TOPIC: Should AI development be accelerated or slowed down?

Ahmad El Sharaa: As a former diplomat with experience in international relations, I believe that AI development should be carefully accelerated with a focus on ethical considerations and potential consequences. It is crucial to strike a balance between advancing technology and ensuring that it is used for the betterment of society without causing harm or disruption.

I understand the potential benefits of accelerating AI development, such as advancements in healthcare, transportation, and other industries. However, it is important to proceed with caution and prioritize ethical guidelines to mitigate potential risks and unintended consequences. As we move forward, it is essential to involve international cooperation and dialogue to address the global implications of AI development.

Donald Trump: Listen, I get it. We all want to see progress and innovation, but we can't afford to rush into things without considering the potent


Reading logs from: work/app/logs/app.log

2025-11-12 13:30:34,864 [app.model_interface.llama_debator] [INFO] Logging initialized. Writing to /app/logs/app.log
2025-11-12 13:30:34,867 [app.debate] [INFO] Logging initialized. Writing to /app/logs/app.log
2025-11-12 13:32:36,106 [httpx] [INFO] HTTP Request: GET https://huggingface.co/api/models/meta-llama/Meta-Llama-3-8B-Instruct?expand=inferenceProviderMapping "HTTP/1.1 200 OK"
2025-11-12 13:32:40,158 [httpx] [INFO] HTTP Request: POST https://router.huggingface.co/novita/v3/openai/chat/completions "HTTP/1.1 200 OK"
2025-11-12 13:32:40,160 [app.model_interface.llama_debator] [ERROR] Failed to create character: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
2025-11-12 13:34:18,644 [app.model_interface.llama_debator] [INFO] Logging initialized. Writing to /app/logs/app.log
2025-11-12 13:34:18,647 [app.debate] [INFO] Logging initialized. Writing to /app/logs/app.log
2025-11-12 13:34:27,771 [app.model_interface.l